<a href="https://colab.research.google.com/github/windopper/NLPDeepLearningPractice/blob/main/EncoderDecoderUsingRNN/Character_Level_Nerual_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시퀀스-투-시퀀스(seq2seq)를 사용하여 문자 레벨 기계 번역기 구현해보기

병렬 코퍼스 데이터에 대한 이해와 전처리

In [1]:
import os
import shutil
import zipfile
import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [7]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [8]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 192341


In [9]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000]
lines.sample(10)

,src,tar
52016,Tom resuscitated Mary.,Tom a réanimé Marie.
21522,Where's your mom?,Où est ta mère ?
1754,Do I snore?,Est-ce que je ronfle ?
21158,We spoke briefly.,Nous nous sommes brièvement entretenues.
19848,See you tomorrow.,On se voit demain.
48649,I have another sister.,J'ai une autre sœur.
36446,It rained yesterday.,Il a plu hier.
32524,We're all students.,Nous sommes tous étudiants.
44620,This is a real steal.,C'est une vraie bonne affaire.
4356,Are you cold?,Avez-vous froid ?


시작 심볼과 종료 심볼 추가

In [10]:
lines.tar = lines.tar.apply(lambda x : '\t '+x+' \n')
lines.sample(10)

,src,tar
21572,Whose turn is it?,\t À qui le tour ? \n
13,Run.,\t Cours ! \n
52680,When did you say that?,\t Quand as-tu dit cela ? \n
36654,Japanese are Asians.,\t Les Japonais sont des Asiatiques. \n
6777,Everyone wins.,\t Tout le monde y gagne. \n
30242,Is the house ready?,\t La maison est-elle prête ? \n
52659,What's your shoe size?,\t Quelle pointure faites-vous ? \n
18720,I still love you.,\t Je t'aime encore. \n
54037,Do you know how to ski?,\t Savez-vous skier ? \n
4732,I admire you.,\t Je t'admire. \n


문자 집합 생성

In [11]:
src_vocab = set()
for line in lines.src:
  for char in line:
    src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

In [12]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print('source 문장의 char 집합 : ', src_vocab_size)
print('target 문장의 char 집합 : ', tar_vocab_size)

source 문장의 char 집합 :  80
target 문장의 char 집합 :  105


리스트로 정렬

In [13]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


문자에 인덱스 부여

In [14]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '°': 76, 'é': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 3

정수 인코딩 진행

In [15]:
encoder_input = []

for line in lines.src:
  encoded_line = []
  for char in line:
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 : ', encoder_input[:5])

source 문장의 정수 인코딩 :  [[30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10], [31, 58, 10]]


In [16]:
decoder_input = []
for line in lines.tar:
  decoded_line = []
  for char in line:
    decoded_line.append(tar_to_index[char])
  decoder_input.append(decoded_line)
print('target 문장의 정수 인코딩 : ', decoder_input[:5])

target 문장의 정수 인코딩 :  [[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [1, 3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 14, 3, 2]]


예측값과 비교하기 위한 실제값 전처리. 시작 심볼에 해당하는 &lt;sos&gt;를 제거

In [17]:
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep>0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :', decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 53, 3, 4, 3, 2], [3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 14, 3, 2]]


모든 데이터에 대하여 패딩 작업 수행

In [18]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 : ', max_src_len)
print('target 문장의 최대 길이 : ', max_tar_len)

source 문장의 최대 길이 :  23
target 문장의 최대 길이 :  76


In [19]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

문자 단위 번역기임으로 워드 임베딩을 사용하지 않고 원-핫 벡터를 사용

In [20]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

디코더 셀의 입력은 이전 디코더 셀의 출력으로 부터 받지만, 모델의 예측이 다른 방향으로 가는 것을 방지하기 위하여 '교사 강요'라는 방법을 쓴다.
decoder_input 값을 새로운 입력으로 사용하여 모델의 예측이 엇나가는 것을 막는다.

seq2seq 기계 번역기 훈련시키기

In [21]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [24]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs는 '교사강요' 방법을 사용할 것이기 때문에 여기서는 불필요하다
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs);

# LSTM은 바닐라 RNN과는 달리 상태가 두 개이다. 은닉상태와 셀 상태
# 자세한 것은 위키독스의 LSTM 파트 참고
encoder_states = [state_h, state_c]

In [25]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 은닉 상태, 셀 상태 전달
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy'
)

model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 53s 57ms/step - loss: 0.7527 - val_loss: 0.6642
Epoch 2/40
750/750 [==============================] - 35s 46ms/step - loss: 0.4572 - val_loss: 0.5326
Epoch 3/40
750/750 [==============================] - 34s 45ms/step - loss: 0.3814 - val_loss: 0.4715
Epoch 4/40
750/750 [==============================] - 35s 47ms/step - loss: 0.3393 - val_loss: 0.4345
Epoch 5/40
750/750 [==============================] - 34s 46ms/step - loss: 0.3108 - val_loss: 0.4114
Epoch 6/40
750/750 [==============================] - 34s 46ms/step - loss: 0.2903 - val_loss: 0.3925
Epoch 7/40
750/750 [==============================] - 34s 46ms/step - loss: 0.2743 - val_loss: 0.3807
Epoch 8/40
750/750 [==============================] - 38s 51ms/step - loss: 0.2615 - val_loss: 0.3712
Epoch 9/40
750/750 [==============================] - 32s 43ms/step - loss: 0.2506 - val_loss: 0.3647
Epoch 10/40
750/750 [==============================] - 32s 43ms/step - loss: 0.241

기계 번역기 동작시키기

전체적인 번역 동작 원리 정리:

<ol>
<li>
번역하고자 하는 입력문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻음
</li>
<li>
상태와 &lt;SOS&gt;에 해당하는 '\t'를 디코더로 보냄
</li>
<li>
디코더가 &lt;EOS&gt;에 해당하는 '\n'이 나올 때까지 다음 문자를 예측하는 행동을 반복
</li>
</ol>

In [26]:
# 인코더 정의
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)




In [27]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [28]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char
    
    # <EOS>에 도달하거나 최대 길이를 넘으면 중단
    if(sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
      stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence



In [29]:
for seq_index in [3, 50, 100, 300, 1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장 :', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장 : Hi.
정답 문장: Salut ! 
번역 문장: Salut. 
-----------------------------------
입력 문장 : I see.
정답 문장: Aha. 
번역 문장: Je suis en train de manger. 
-----------------------------------
입력 문장 : Hug me.
정답 문장: Serrez-moi dans vos bras ! 
번역 문장: Serre-moi dans tes bras ! 
-----------------------------------
입력 문장 : Help me.
정답 문장: Aidez-moi. 
번역 문장: Aide-moi. 
-----------------------------------
입력 문장 : I am sure.
정답 문장: Je suis sûr. 
번역 문장: Je suis tendu. 
